In [1]:
import numpy as np
import pandas as pd
from glob import glob
from os.path import join
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm

import torch
import torch.nn as nn
import torchvision
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from torchvision import models
import torch.optim as optim
from torch.utils.data import DataLoader

class AgeDataset(torch.utils.data.Dataset):
    def __init__(self, data_path, annot_path, train=True):
        super(AgeDataset, self).__init__()
        self.annot_path = annot_path
        self.data_path = data_path
        self.train = train

        self.ann = pd.read_csv(annot_path)
        self.files = self.ann['file_id']
        if train:
            self.ages = self.ann['age']
        self.transform = self._transform(224)

    @staticmethod
    def _convert_image_to_rgb(image):
        return image.convert("RGB")

    def _transform(self, n_px):
        mean = [0.485, 0.456, 0.406]
        std = [0.229, 0.224, 0.225]
        return Compose([
            Resize(n_px),
            self._convert_image_to_rgb,
            ToTensor(),
            Normalize(mean, std),
        ])

    def read_img(self, file_name):
        im_path = join(self.data_path, file_name)
        img = Image.open(im_path)
        img = self.transform(img)
        return img

    def __getitem__(self, index):
        file_name = self.files[index]
        img = self.read_img(file_name)
        if self.train:
            age = self.ages[index]
            return img, age
        else:
            return img

    def __len__(self):
        return len(self.files)

train_path = '/kaggle/input/smai-24-age-prediction/content/faces_dataset/train'
train_ann = '/kaggle/input/smai-24-age-prediction/content/faces_dataset/train.csv'
dataset = AgeDataset(train_path, train_ann, train=True)
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [0.8, 0.2])

test_path = '/kaggle/input/smai-24-age-prediction/content/faces_dataset/test'
test_ann = '/kaggle/input/smai-24-age-prediction/content/faces_dataset/submission.csv'
test_dataset = AgeDataset(test_path, test_ann, train=False)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

@torch.no_grad
def predict(loader, model):
    model.eval()
    predictions = []
    for img in tqdm(loader):
        img = img.to(device)
        pred = model(img)
        predictions.extend(pred.flatten().detach().tolist())
    return predictions

def validation_loop(val_loader, model, criterion):
    model.eval()
    running_val_loss = 0.0
    with torch.no_grad():
        for images, ages in tqdm(val_loader):
            images = images.to(device)
            ages = ages.to(device).float().unsqueeze(1)  # Ensure age is a float and has correct shape
            outputs = model(images)
            loss = criterion(outputs, ages)
            running_val_loss += loss.item() * images.size(0)
    print(f"Val Loss: {running_val_loss / len(val_loader.dataset)}")

def train_model(model, train_loader, val_loader, test_loader, criterion, optimizer, num_epochs=5):
    for epoch in range(num_epochs):
        train_running_loss = 0.0
        model.train()
        for images, ages in tqdm(train_loader):
            images = images.to(device)
            ages = ages.to(device).float().unsqueeze(1)  # Ensure age is a float and has correct shape
            optimizer.zero_grad()  # Zero the parameter gradients
            outputs = model(images)
            loss = criterion(outputs, ages)
            loss.backward()  # Backpropagate the loss
            optimizer.step()  # Optimize the weights
            train_running_loss += loss.item() * images.size(0)
        epoch_loss = train_running_loss / len(train_loader.dataset)
        print(f"Epoch {epoch + 1}, Training Loss: {epoch_loss:.4f}, ", end="")
        validation_loop(val_loader, model, criterion)
        preds = predict(test_loader, model)
        submit = pd.read_csv('/kaggle/input/smai-24-age-prediction/content/faces_dataset/submission.csv')
        submit['age'] = preds
        submit.to_csv(f'/kaggle/working/submission_epoch_{epoch + 1}.csv', index=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet34(pretrained=True)

num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 1)
)  # Assuming age prediction is a regression task

for name, param in model.named_parameters():
    if "layer" in name:
        if "layer1" in name:
            param.requires_grad = False
    elif 'fc.' not in name:
        param.requires_grad = False

model = model.to(device)

# Setup loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Call the train_model function
train_model(model, train_loader, val_loader, test_loader, criterion, optimizer, num_epochs=40)

# Create submission CSV file
preds = predict(test_loader, model)
submit = pd.read_csv('/kaggle/input/smai-24-age-prediction/content/faces_dataset/submission.csv')
submit['age'] = preds
submit.head()
submit.to_csv('/kaggle/working/submission.csv', index=False)


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 89.7MB/s]
100%|██████████| 267/267 [02:12<00:00,  2.02it/s]


Epoch 1, Training Loss: 551.3201, 

100%|██████████| 67/67 [00:27<00:00,  2.45it/s]


Val Loss: 256.0041039421275


100%|██████████| 267/267 [01:18<00:00,  3.39it/s]


...

100%|██████████| 267/267 [01:18<00:00,  3.39it/s]


Epoch 40, Training Loss: 1.1236, 

100%|██████████| 67/67 [00:15<00:00,  4.30it/s]


Val Loss: 48.67746387113448


100%|██████████| 31/31 [00:07<00:00,  4.42it/s]
